In [1]:
import pandas as pd
import numpy as np
import os
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, FloatType
from pyspark.sql import Row

2023-09-30 14:28:48.326321: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-30 14:28:48.367186: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-09-30 14:28:48.367302: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-09-30 14:28:48.367375: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-09-30 14:28:48.375003: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-30 14:28:48.376174: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

# HandLandmarker Detector

In [2]:
# Create an HandLandmarker object.
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


# Data processing

In [3]:
#Create empty DatFrame
df = pd.DataFrame(columns = [f"x{i}" for i in range(21)] + [f"y{i}" for i in range(21)] + [f"z{i}" for i in range(21)])
df

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,z11,z12,z13,z14,z15,z16,z17,z18,z19,z20


In [4]:
# Function to recursively read files in folders
def read_images(folder_path):
    X = []
    y = []
    classes = set([])
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            
            file_name = file
            file_path = root
            
            image = mp.Image.create_from_file(file_path + "/" + file_name)
            
            detection_result = detector.detect(image)
            letter = file_path.split("/")[-1]
            if(len(detection_result.hand_landmarks) == 1 and letter != "Blank"):
                X.append(detection_result.hand_landmarks)
                y.append(letter)
                classes.add(letter)
            
    return X, y

In [5]:
X_train, y_train = read_images("./data/Train_Alphabet/")

In [6]:
X_test, y_test = read_images("./data/Test_Alphabet/")

In [9]:
def image_to_df(samples, df):
    samples = np.array(samples)
    samples = [sample.flatten() for sample in samples]
    for i in range(len(samples)):
        row = {}
        for j in range(len(samples[i])):
            row[f"x{j}"] = samples[i][j].x
            row[f"y{j}"] = samples[i][j].y
            row[f"z{j}"] = samples[i][j].z
        df.loc[len(df)] = row

In [10]:
image_to_df(X_train, df)
image_to_df(X_test, df)

In [11]:
y = np.append(y_train, y_test, axis=0)
df["class"] = y

In [12]:
df

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,z12,z13,z14,z15,z16,z17,z18,z19,z20,class
0,0.488708,0.404476,0.374821,0.404135,0.455417,0.412926,0.400308,0.395657,0.398848,0.465933,...,0.019789,0.019715,-0.029707,-0.042078,-0.034466,-0.010245,-0.060913,-0.075666,-0.076282,U
1,0.501474,0.418070,0.386350,0.419969,0.469452,0.380163,0.374764,0.375448,0.382550,0.442257,...,-0.021458,0.033095,-0.033578,-0.057040,-0.051505,0.020642,-0.052230,-0.078819,-0.081988,U
2,0.495314,0.587333,0.620176,0.576634,0.526020,0.654865,0.649304,0.642526,0.637820,0.586860,...,0.020883,0.048545,-0.020828,-0.062715,-0.064058,0.044152,-0.037272,-0.072273,-0.079230,U
3,0.505301,0.410870,0.376945,0.414208,0.478322,0.362417,0.348918,0.344424,0.347965,0.436176,...,-0.010602,0.034612,-0.048979,-0.076620,-0.067791,0.026952,-0.070757,-0.108855,-0.115611,U
4,0.477609,0.533127,0.555380,0.521581,0.488303,0.646072,0.648374,0.645726,0.645272,0.584262,...,-0.036960,0.042070,-0.034447,-0.078379,-0.079646,0.053739,-0.024867,-0.062179,-0.070469,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23191,0.477049,0.567099,0.608463,0.564787,0.504941,0.573284,0.519666,0.477483,0.433821,0.485131,...,-0.217575,-0.128351,-0.173626,-0.164545,-0.155020,-0.133203,-0.162356,-0.151179,-0.140023,Q
23192,0.522720,0.617993,0.678742,0.705613,0.744512,0.647478,0.681218,0.704073,0.709654,0.558177,...,-0.291892,-0.293102,-0.331692,-0.255129,-0.208393,-0.316775,-0.342012,-0.300316,-0.272283,Q
23193,0.460417,0.409762,0.403771,0.453101,0.502669,0.466622,0.502520,0.536604,0.564446,0.528612,...,-0.197802,-0.095953,-0.147584,-0.156183,-0.155339,-0.089400,-0.128006,-0.137008,-0.138630,Q
23194,0.453195,0.532862,0.581763,0.550798,0.492235,0.541501,0.488201,0.454140,0.426474,0.456165,...,-0.278054,-0.158420,-0.230677,-0.229247,-0.218678,-0.163200,-0.210383,-0.208543,-0.201904,Q


In [13]:
df.to_csv("data/asl_dataset.csv")